In [1]:
# !pip install keras-tuner

In [1]:
import numpy as np
from tensorflow import keras

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.4.1'

In [4]:
fashion = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels),(test_images, test_labels) = fashion.load_data()

## Scaling the images

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images.shape

(60000, 28, 28)

In [8]:
test_images.shape

(10000, 28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
train_images.shape

(60000, 28, 28, 1)

## Defining a function to create tuned model 

In [10]:
# keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1) )

In [11]:
import numpy as np
np.arange(32,128+16,16)

array([ 32,  48,  64,  80,  96, 112, 128])

In [12]:
def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D(pool_size=(2, 2) ),
        
    keras.layers.Flatten(),
        
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [22]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [23]:
tuner_search = RandomSearch(build_model, 
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='Mnist Fashion')

In [24]:
# this is the main Model training step!!!
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 06m 05s]
val_accuracy: 0.9150000214576721

Best val_accuracy So Far: 0.9150000214576721
Total elapsed time: 00h 15m 00s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |48                
conv_1_kernel     |3                 |5                 
conv_2_filter     |48                |48                
conv_2_kernel     |3                 |3                 
dense_1_units     |48                |112               
learning_rate     |0.01              |0.001             

Epoch 1/3
1688/1688 [==============================] - 203s 120ms/step - loss: 0.5531 - accuracy: 0.8026 - val_loss: 0.3152 - val_accuracy: 0.8853
Epoch 2/3
 522/1688 [========>.....................] - ETA: 2:18 - loss: 0.3086 - accuracy: 0.8858

KeyboardInterrupt: 

In [21]:
tuner_search.get_best_models(num_models=1)[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 80)                368720    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 402,938
Trainable params: 402,938
Non-trainable params: 0
__________________________________________________

In [32]:
model = tuner_search.get_best_models(num_models=1)[0]

In [33]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 311s 184ms/step - loss: 0.2016 - accuracy: 0.9259 - val_loss: 0.2358 - val_accuracy: 0.9148
Epoch 5/10
1688/1688 [==============================] - 281s 167ms/step - loss: 0.1723 - accuracy: 0.9359 - val_loss: 0.2568 - val_accuracy: 0.9105
Epoch 6/10
1688/1688 [==============================] - 287s 170ms/step - loss: 0.1482 - accuracy: 0.9440 - val_loss: 0.2500 - val_accuracy: 0.9130
Epoch 7/10
1688/1688 [==============================] - 282s 167ms/step - loss: 0.1264 - accuracy: 0.9530 - val_loss: 0.3057 - val_accuracy: 0.9032
Epoch 8/10
1688/1688 [==============================] - 280s 166ms/step - loss: 0.1123 - accuracy: 0.9571 - val_loss: 0.2673 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 292s 173ms/step - loss: 0.0920 - accuracy: 0.9658 - val_loss: 0.2670 - val_accuracy: 0.9192
Epoch 10/10
1688/1688 [==============================] - 286s 169ms/step - loss: 0.0813 - accuracy: 0.9700 - val